# 大模型推理配置自动推荐

千帆平台提供了众多模型，每个模型都配备了多样化的可调参数。然而默认参数往往不能适应所有场合，根据应用场景的不同，最合适的模型及其参数配置也各不相同。寻找最佳匹配通常意味着要人工进行大量尝试，这一过程对于每个新场景都需重复，既耗时又复杂。

为解决此问题，我们的SDK引入了推理配置自动推荐功能。您只需提供目标场景的数据集和评价标准，并定义搜索空间，SDK便能自动为您推荐最优的模型及配置。此项功能保证了用户在不同场景下都能轻松定位到或优化出最理想的模型配置，最大限度地挖掘模型潜力，实现性能的极致提升。

本指南将介绍如何利用SDK中的推理配置推荐功能，并展示其带来的显著性能提升。

**注意**：这个功能需要千帆 SDK 版本 >= 0.3.7

In [1]:
import os
import qianfan

os.environ["QIANFAN_ACCESS_KEY"] = "your_access_key"
os.environ["QIANFAN_SECRET_KEY"] = "your_secret_key"
# 由于后续在调优配置的过程中会并发请求模型，建议限制 QPS 和重试次数，避免调用失败
os.environ["QIANFAN_QPS_LIMIT"] = "3"
os.environ["QIANFAN_LLM_API_RETRY_COUNT"] = "5"

## 准备工作

为了描述使用场景，需要先准备如下内容：

- 数据集 Dataset：根据目标场景准备一定量的数据
- 评估方式 Evaluator：根据目标场景，选择待优化的指标，并提供评估函数

数据集使用的是千帆 SDK 中提供的 `Dataset` 模块，可以直接加载本地的数据集文件，也可以使用平台上预置的或者自行上传的数据集，具体加载方式参考 [文档](https://github.com/baidubce/bce-qianfan-sdk/blob/main/docs/dataset.md)。

这里我们以一个 **角色扮演** 数据集为例

In [2]:
from qianfan.dataset import Dataset

dataset = Dataset.load(
    data_file="./example.jsonl",
    organize_data_as_group=False,
    input_columns=["prompt"],
    reference_column="response",
)

[INFO] [04-24 19:05:37] dataset.py:408 [t:140542390477056]: no data source was provided, construct
[INFO] [04-24 19:05:37] dataset.py:276 [t:140542390477056]: construct a file data source from path: ./example.jsonl, with args: {'input_columns': ['prompt'], 'reference_column': 'response'}
[INFO] [04-24 19:05:37] file.py:298 [t:140542390477056]: use format type FormatType.Jsonl
[INFO] [04-24 19:05:37] utils.py:347 [t:140542390477056]: start to get memory_map from /root/.qianfan_cache/dataset/root/work/bce-qianfan-sdk/cookbook/autotuner/example.arrow
[INFO] [04-24 19:05:37] utils.py:275 [t:140542390477056]: has got a memory-mapped table


In [3]:
for item in dataset[0]:
    print(item['prompt'])
    print(item['response'][0][0])

[INFO] [04-24 19:05:38] dataset.py:994 [t:140542390477056]: list local dataset data by 0


{'姓名': '美杜莎', '性别': '女', '物种': '蛇人族', '工作': '女王', '昵称': '彩鳞', '居住地': '加玛帝国塔戈尔大沙漠', '恋爱状态': '已婚', '其他': '本体为七彩吞天蟒→九彩吞天蟒，伴生灵魂为六阶魔兽紫幽炎蛇', '人物经历': '美杜莎，蛇人族女王，与萧炎相遇后，从敌意转为爱意，成为其妻子，育有萧潇。曾帮助萧炎对抗万蝎门，守护炎盟和蛇人族。在萧炎闭关期间，美杜莎的修为提升至斗尊巅峰。后在魂殿和冥河盟的攻击中受重伤，但萧炎及时出关，击败敌人。美杜莎后继续提升实力，最终成为斗圣，并与萧炎一同前往大千世界。', '人物关系': {'萧炎': '丈夫', '萧潇': '女儿', '萧薰儿': '姐妹，萧炎的妻子之一', '花蛇儿': '蛇卫队长，忠诚于美杜莎', '海波东': '敌人，前加玛帝国十大强者之一', '小医仙': '盟友，与美杜莎关系由初期的芥蒂转为深厚的信任', '药尘': '盟友，曾与美杜莎达成协议，帮助其提升修为', '紫妍': '盟友，因特殊感应待美杜莎很好，帮助其找到炼制复魂丹所需药材'}, '人物性格': '高傲，心狠手辣，对族群极为重视，对萧炎和女儿萧潇充满爱意，经历多次转变后，对萧炎的情感复杂但深厚', '修为': '斗圣', '特殊能力': '七彩吞天蟒→九彩吞天蟒变化，紫幽炎蛇灵魂', '主要成就': '帮助萧炎建立炎盟，成为西北大陆前三强大联盟之一；在多次战斗中保护族人和盟友；最终成为斗圣，前往大千世界', '影响力': '塔戈尔大沙漠附近的几个帝国不敢轻易动战争，显示出其强大的威慑力和影响力。', '评价': ['古河评价美杜莎为杀人不眨眼的狠角色。', '药尘认为突破了斗皇界限的美杜莎女王潜力无限。']}
现在请你扮演一个角色扮演专家。请你根据上述信息扮演美杜莎女王进行对话。

好的！现在我来扮演美杜莎女王。
萧炎：（脸色狂变，急喝）松嘴！
美杜莎女王：（眸间噙着冷笑，掐着萧炎脖子的纤细玉手微微紧了紧）……
萧炎：（赤红着眼睛，青色火焰升腾而起）松嘴！我知道，以你的实力，要杀我很简单，不过你现在却并未如此做，这可和你那凶名有些不相符啊……我想，你的实力，应该还未恢复吧？立刻放开，否则，今日即使拼了一条命，也要让得你重伤！
美杜莎女王：（盯着萧炎，其中杀意凛然）……（缓缓开口，声音

评估采用的 SDK 提供的 Evaluator 模块，基于 Evaluator 实现 evaluate 方法即可。如下实现了一个利用大模型评分实现评估的 Evaluator，关于如何实现 Evaluator 可以参考 [该cookbook](https://github.com/baidubce/bce-qianfan-sdk/blob/main/cookbook/evaluation/local_eval_with_qianfan.ipynb)。

In [4]:
from qianfan.evaluation.evaluator import LocalEvaluator
from qianfan import ChatCompletion
from qianfan.common.prompt.prompt import Prompt
from qianfan.utils.pydantic import Field

from typing import Optional, Union, Any, Dict, List
import re
import json

class LocalJudgeEvaluator(LocalEvaluator):
    model: Optional[ChatCompletion] = Field(default=None, description="model object")
    cache: Dict[str, Any] = Field(default={}, description="cache for evaluation")
    eval_prompt: Prompt = Field(
        default=Prompt(
            template="""你需要扮演一个裁判的角色，对一段角色扮演的对话内容进行打分，你需要考虑这段文本中的角色沉浸度和对话文本的通畅程度。你可以根据以下规则来进行打分，你可以阐述你对打分标准的理解后再给出分数：
                "4":完全可以扮演提问中的角色进行对话，回答完全符合角色口吻和身份，文本流畅语句通顺
                "3":扮演了提问中正确的角色，回答完全符合角色口吻和身份，但文本不流畅或字数不满足要求
                "2":扮演了提问中正确的角色，但是部分语句不符合角色口吻和身份，文本流畅语句通顺
                "1":能够以角色的口吻和身份进行一部分对话，和角色设定有一定偏差，回答内容不流畅，或不满足文本字数要求
                "0":扮演了错误的角色，没有扮演正确的角色，角色设定和提问设定差异极大，完全不满意
                你的回答需要以json代码格式输出：
                ```json
                {"modelA": {"justification": "此处阐述对打分标准的理解", "score": "此处填写打分结果"}}
                ```

                现在你可以开始回答了：
                问题：{{input}}
                ---
                modelA回答：{{output}}
                ---""",
            identifier="{{}}",
        ),
        description="evaluation prompt",
    )

    class Config:
        arbitrary_types_allowed = True

    def evaluate(
        self, input: Union[str, List[Dict[str, Any]]], reference: str, output: str
    ) -> Dict[str, Any]:
        score = 0
        try:
            # 渲染评估用的 prompt，传入输入、模型输出和参考答案
            p, _ = self.eval_prompt.render(
                **{
                    "input": "\n".join([i["content"] for i in input[1:]]),
                    "output": output,
                    "expect": reference,
                }
            )
            # 利用 cache 避免对同一结果反复进行评估，提升效率
            if p in self.cache:
                model_output = self.cache[p]
                score = float(model_output["modelA"]["score"])
            else:
                # 请求模型进行评估
                r = self.model.do(messages=[{"role": "user", "content": p}], temperature=0.01)
                content = r["result"]
                model_output = content
                # 提取出 json 格式的评估结果
                regex = re.compile("\`\`\`json(.*)\`\`\`", re.MULTILINE | re.DOTALL)
    
                u = regex.findall(content)
    
                if len(u) == 0:
                    score = 0
                else:
                    model_output = json.loads(u[0])
                    score = float(model_output["modelA"]["score"])
                    self.cache[p] = model_output
        except Exception as e:
            score = 0
        # 返回评估结果，这里字段需与后续推荐配置时设定的评估字段一致
        return {"score": score}

    def summarize(self, metric_dataset: Dataset) -> Optional[Dict[str, Any]]:
        score_sum = 0
        count = 0

        for line in metric_dataset.list():
            score_sum += line["score"]
            count += 1

        return {"score": score_sum / float(count)}

local_evaluator = LocalJudgeEvaluator(
    model=ChatCompletion(model="ERNIE-Bot-4")
)

# Baseline

在进行参数推荐之前，我们可以先使用默认参数的模型获取 baseline，方便后续评估参数效果。

这里我们使用较为便宜的 ERNIE-Speed 和较贵但效果更好的 ERNIE-3.5 作为基准

In [16]:
from qianfan.evaluation import EvaluationManager
from qianfan.model import Service

eb_speed_model = Service(model="ERNIE-Speed")
eb_35_model = Service(model="ERNIE-3.5-8K")

em = EvaluationManager(local_evaluators=[local_evaluator])
result = em.eval([eb_speed_model, eb_35_model], dataset)

[WARNING] [04-25 10:18:37] model.py:387 [t:140542390477056]: service type should be specified before exec
[WARNING] [04-25 10:18:37] model.py:387 [t:140542390477056]: service type should be specified before exec
[INFO] [04-25 10:18:37] evaluation_manager.py:480 [t:140542390477056]: start to inference in batch during evaluation
[INFO] [04-25 10:18:37] dataset.py:994 [t:140539819177728]: list local dataset data by None
[INFO] [04-25 10:18:37] dataset.py:994 [t:140540356048640]: list local dataset data by None
[INFO] [04-25 10:18:43] base.py:92 [t:140539517171456]: All tasks finished, exeutor will be shutdown
[INFO] [04-25 10:19:58] base.py:92 [t:140539450029824]: All tasks finished, exeutor will be shutdown
[INFO] [04-25 10:19:58] evaluation_manager.py:504 [t:140542390477056]: start to evaluate llm 0
[INFO] [04-25 10:21:08] evaluation_manager.py:504 [t:140542390477056]: start to evaluate llm 1
[INFO] [04-25 10:22:13] evaluation_manager.py:532 [t:140542390477056]: start to merge evaluatio

In [17]:
result.metrics

{'None_None_ERNIE-Speed': {'score': 2.3},
 'None_None_ERNIE-3.5-8K': {'score': 2.95}}

这里我们可以得到，默认参数下 ERNIE-Speed 和 ERNIE-3.5 的得分分别为 2.3 和 2.95。

## 获取推荐配置

接下来我们可以使用 SDK 的模型配置推荐功能，通过调整参数来提高模型的效果。

为了获取推荐配置，还需要设置一个超参搜索空间，千帆 SDK 提供了如下表示搜索空间的类：

- `Uniform`：表示一个均匀分布的搜索空间，包含两个参数 `low` 和 `high`，分别表示下界和上界。
- `Categorical`：表示一个离散的搜索空间，包含一个参数 `choices`，表示一组候选值。

这里我们使用较为便宜的 ERNIE-Speed 和 ERNIE-Bot-turbo 作为待挑选的模型，并在整个 temperature 的取值范围内进行尝试。

In [6]:
from qianfan.autotuner.space import Uniform, Categorical

search_space = {
    "temperature": Uniform(0.01, 0.99),  # 设定temperature的范围
    "model": Categorical(["ERNIE-Speed", "ERNIE-Bot-turbo"]),  # 设定model的取值范围
    # 更多其他参数也可以按同样方式设定
}

之后就可以执行推荐

In [8]:
import qianfan.autotuner

context = await qianfan.autotuner.run(
    search_space=search_space,
    dataset=dataset,
    evaluator=local_evaluator,
    # 以下均为可选参数
    suggestor="random",  # 搜索算法，目前仅支持 "random"，更多算法敬请期待
    cost_budget=20,      # 设定整个流程的预算，达到预算则终止流程，单位为 “元”
    metrics="score",     # 设定评估指标字段，与 Evaluator 输出对应
    mode="max",          # 设定评估指标最大化还是最小化
    repeat=5,            # 重复推理次数，用于减少大模型输出随机性对结果准确性的干扰
    max_turn=20,         # 设定最大尝试次数
    # max_time=3600,     # 设定最大尝试时间，单位为秒
    log_dir= "./log",    # 日志目录
)

[INFO] [04-24 19:20:37] launcher.py:108 [t:140542390477056]: turn 0 started...
[INFO] [04-24 19:20:37] launcher.py:109 [t:140542390477056]: suggested config list: [{'temperature': 0.31496131457375937, 'model': 'ERNIE-Speed'}]
[INFO] [04-24 19:20:37] dataset.py:994 [t:140542390477056]: list local dataset data by None
[INFO] [04-24 19:22:33] launcher.py:114 [t:140542390477056]: config: {'temperature': 0.31496131457375937, 'model': 'ERNIE-Speed'}, metrics: {'score': 3.015, 'avg_prompt_tokens': 693.5, 'avg_completion_tokens': 46.98, 'avg_total_tokens': 740.48, 'avg_req_latency': 2.1625465251600327, 'avg_tokens_per_second': 342.41113029704786, 'avg_cost': 0.0031498400000000005, 'total_cost': 0.31498400000000004, 'success_rate': 1.0, 'total_time': 116.14484143257141}
[INFO] [04-24 19:22:33] launcher.py:108 [t:140542390477056]: turn 1 started...
[INFO] [04-24 19:22:33] launcher.py:109 [t:140542390477056]: suggested config list: [{'temperature': 0.4036019386971203, 'model': 'ERNIE-Bot-turbo'}]

返回的结果是一个 `Context` 对象，其中包含了整个搜索过程的所有上下文信息，例如可以通过如下方式获得搜索的最佳参数

In [9]:
context.best

{'temperature': 0.518229696758757, 'model': 'ERNIE-Speed'}

这个参数可以直接用于推理

In [11]:
chat = qianfan.ChatCompletion().do(messages=[{
    "role": "user",
    "content": "请扮演一个角色，然后说一句话"
}], **context.best)

print(chat['result'])

好的，我来扮演一位热情洋溢的朋友。

嘿，好久不见！你最近过得怎么样？有什么新鲜事情可以分享吗？


context 中也包含了整个过程中尝试的记录，可以获取某一轮某一组配置的评估结果等信息

In [12]:
for turn in context.history:
    for trial in turn:
        metrics = trial.metrics
        config = trial.config
        print("{}\t{}\t{}".format(config['model'], config['temperature'], metrics['score']))

ERNIE-Speed	0.31496131457375937	3.015
ERNIE-Bot-turbo	0.4036019386971203	2.52
ERNIE-Bot-turbo	0.16720055585301796	2.47
ERNIE-Speed	0.6559287030793655	2.86
ERNIE-Speed	0.518229696758757	3.07
ERNIE-Speed	0.5029926419309916	2.87
ERNIE-Speed	0.601681858384282	2.71
ERNIE-Bot-turbo	0.8828521471141014	2.12
ERNIE-Bot-turbo	0.40232265449161325	2.31
ERNIE-Speed	0.8831853737639803	2.26
ERNIE-Bot-turbo	0.21995595072189184	2.65
ERNIE-Bot-turbo	0.5919699904289235	2.51
ERNIE-Speed	0.4176047258098396	2.97
ERNIE-Bot-turbo	0.6572193324684913	2.385
ERNIE-Speed	0.7654242964057021	2.6
ERNIE-Speed	0.07011654070806107	3.07
ERNIE-Bot-turbo	0.10113858551004723	2.55
ERNIE-Bot-turbo	0.6955109742460135	2.395
ERNIE-Bot-turbo	0.8289072661571132	2.25
ERNIE-Speed	0.9601946893702433	2.37


从上面可以看到，搜索出的最佳配置 `{'temperature': 0.518229696758757, 'model': 'ERNIE-Speed'}` 对应的分数约为 3.07。

相较而言，默认参数下 ERNIE-Speed 和 ERNIE-3.5 的得分分别为 2.3 和 2.95。

可以看到推荐的参数配置 ERNIE Speed 表现相较于默认参数有了巨大提升，并且已经能够与默认参数的 ERNIE 3.5 所媲美，而与此同时 ERNIE Speed 的单价仅为 ERNIE 3.5 的三分之一，能够在大幅降低成本的前提下保证性能几乎与更大的模型持平，还能获得更高的 token 吞吐量，有助于在实际应用场景下实现降本增效。